### SAのテンプレート

In [1]:
ex_num = "006"

In [2]:
import os
# 環境によって変更
# os.chdir('/home/jovyan/work/notebook')

path = "../../output"
# path = "../model"

# 基準にするベースファイル
base_file_path = "../input/sub20241219.csv"

In [3]:
class CFG:
    path = "../model"
    base_file_path = "../input/sub20241219.csv"
    BATCH_SIZE = 32

In [4]:
import numpy as np
import pandas as pd
import os
import time
import csv

from tqdm.notebook import tqdm

import random
from typing import List, Callable
import itertools, math
from typing import List, Tuple

from utils import PerplexityCalculator

### データ読み込み

In [5]:
df = pd.read_csv(CFG.base_file_path)

In [6]:
sub_df = df.copy()
sub_df["best_value"] = 0

In [7]:
df

,id,text
0,0,reindeer mistletoe elf gingerbread family adve...
1,1,reindeer sleep walk the night and drive mistle...
2,2,jingle yuletide carol cheer grinch holiday hol...
3,3,sleigh the holly jingle unwrap gifts of magi r...
4,4,peppermint candy milk chocolate eggnog fruitca...
5,5,sleigh of naughty nice the of not the to and t...


### 関数

In [8]:
# 2単語組み合わせ全てから最低スコアとなるペアを探す
def find_initial_pair(
    CFG,
    scorer,
    words
):
    best_score = float('inf')
    best_pair = None
    for i in range(len(words)):
        perms = []
        for j in range(i+1, len(words)):
            pair = [words[i], words[j]]
            perms.append(" ".join(pair))
        
        scores = scorer.get_perplexity(perms, batch_size=CFG.BATCH_SIZE)
        for j, perm in enumerate(perms):
            if scores[j] < best_score:
                best_score = scores[j]
                best_pair = perm.split(" ")
                
                # print(f"best_score: {best_score}, best_pair: {best_pair}")
     
    return best_pair, best_score

In [9]:
# 文字列のファイルからコストを最小化する文字列を生成する
def greedy_add_initialize(
    CFG,
    scorer,
    initial_words: list
):
    
    # 全単語
    candidates = initial_words[:]

    print("candidates:", candidates)
    print("-----初期配置の組み合わせ計算開始-----")
    # 初期配置を求める
    initial, initial_score = find_initial_pair(
        CFG,
        scorer,
        candidates
    )
    print("-----初期配置の組み合わせ計算終了-----")
    for w in initial:
        candidates.remove(w)

    print("initial:", initial, initial_score)
    current_order = initial[:]

    print("-----貪欲法での挿入開始-----")
    # 貪欲に残りを挿入
    while candidates:
        best_score = float('inf')
        best_word = None
        best_pos = None

        # 残っている単語それぞれについて
        for w in candidates:
            # current_orderのあらゆる挿入位置を試す
            # 挿入位置は0からlen(current_order)まで
            
            perms = []
            for pos in range(len(current_order) + 1):
                new_order = current_order[:pos] + [w] + current_order[pos:]
                perms.append(" ".join(new_order))
                
            scores = scorer.get_perplexity(perms, batch_size=CFG.BATCH_SIZE)
                
            for pos in range(len(current_order) + 1):
                if scores[pos] < best_score:
                    best_score = scores[pos]
                    best_word = w
                    best_pos = pos
                    
                    # print("best_word:", best_word, best_score)
        
        # best_wordをbest_posに挿入
        current_order.insert(best_pos, best_word)
        candidates.remove(best_word)
        print("current_order:", current_order, best_score)
    print("-----貪欲法での挿入終了-----")
    
    best_score = scorer.get_perplexity([" ".join(current_order)], batch_size=1)[0]
    
    return current_order, best_score

In [10]:
def search_optimal_permutation(
    CFG,
    id: int, 
    scorer, 
    max_iterations: int = 1000, 
    BATCH_SIZE=64
):
    
    df = pd.read_csv(CFG.base_file_path)
    
    start = time.time()
    # 文字列を単語に分割
    words = df.loc[id,"text"].split()
    
    initial_sequence, initial_score = greedy_add_initialize(
        CFG,
        scorer=scorer,
        initial_words=words
    )
    
    print(f"{id}th sample: {initial_sequence}")
    print(f"Elapsed time: {time.time() - start:.2f} sec")
    print(f"initial_score: {initial_score}")
    
    
    best_sequence = initial_sequence
    best_score = initial_score
    
    return best_sequence, best_score


In [11]:
# 文字列型にのみダブルクォーテーションを付ける関数
def add_quotes_to_strings(value):
    if isinstance(value, str):  # 文字列型の場合
        return f'"{value}"'
    return value  # それ以外はそのまま

In [12]:
# LOAD GEMMA SCORER
scorer = PerplexityCalculator(f'{path}/gemma_2_9b')

cuda


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [13]:
# scorer.clear_gpu_memory()

In [14]:
for i in [0, 1, 2, 3, 4, 5]:
    
    print(f"Processing {i}th sample")
    best_sequence, best_score = search_optimal_permutation(
        CFG,
        id=i,
        scorer=scorer,
        max_iterations=2**10,
    )
    
    print("Best sequence:", best_sequence)
    print("Best score:", best_score)
    sub_df.loc[i, "best_value"] = best_score
    sub_df.loc[i, "text"] = " ".join(best_sequence)



Processing 0th sample
candidates: ['reindeer', 'mistletoe', 'elf', 'gingerbread', 'family', 'advent', 'scrooge', 'chimney', 'fireplace', 'ornament']
-----初期配置の組み合わせ計算開始-----
-----初期配置の組み合わせ計算終了-----
initial: ['reindeer', 'scrooge'] 7671.845667278888
-----貪欲法での挿入開始-----
current_order: ['reindeer', 'mistletoe', 'scrooge'] 1659.1529831272167
current_order: ['reindeer', 'mistletoe', 'scrooge', 'elf'] 1026.1751780534337
current_order: ['reindeer', 'mistletoe', 'scrooge', 'gingerbread', 'elf'] 874.3121824177148
current_order: ['reindeer', 'mistletoe', 'scrooge', 'gingerbread', 'elf', 'ornament'] 750.765798300995
current_order: ['reindeer', 'mistletoe', 'scrooge', 'gingerbread', 'chimney', 'elf', 'ornament'] 686.2554949467076
current_order: ['reindeer', 'mistletoe', 'scrooge', 'gingerbread', 'chimney', 'fireplace', 'elf', 'ornament'] 555.7466361419574
current_order: ['reindeer', 'mistletoe', 'scrooge', 'gingerbread', 'chimney', 'fireplace', 'elf', 'ornament', 'advent'] 544.9975231641602
curre

/tmp/ipykernel_114/685970413.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '551.4217815078388' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  sub_df.loc[i, "best_value"] = best_score


-----初期配置の組み合わせ計算終了-----
initial: ['the', 'scrooge'] 4074.4864777337
-----貪欲法での挿入開始-----
current_order: ['the', 'scrooge', 'family'] 1302.284097746097
current_order: ['and', 'the', 'scrooge', 'family'] 919.8574845998919
current_order: ['reindeer', 'and', 'the', 'scrooge', 'family'] 473.50205375098
current_order: ['reindeer', 'and', 'mistletoe', 'the', 'scrooge', 'family'] 408.18428444915037
current_order: ['reindeer', 'and', 'mistletoe', 'the', 'scrooge', 'family', 'advent'] 534.4563168515505
current_order: ['reindeer', 'and', 'mistletoe', 'the', 'scrooge', 'family', 'advent', 'ornament'] 612.7571861239999
current_order: ['reindeer', 'and', 'mistletoe', 'the', 'scrooge', 'family', 'advent', 'fireplace', 'ornament'] 642.1640174789829
current_order: ['reindeer', 'and', 'mistletoe', 'the', 'elf', 'scrooge', 'family', 'advent', 'fireplace', 'ornament'] 624.8427387029609
current_order: ['reindeer', 'and', 'mistletoe', 'the', 'elf', 'scrooge', 'gingerbread', 'family', 'advent', 'fireplace', 

In [15]:
sub_df

,id,text,best_value
0,0,reindeer mistletoe scrooge gingerbread chimney...,551.421782
1,1,reindeer and mistletoe the elf scrooge gingerb...,667.744924
2,2,jingle yuletide carol grinch magi holiday chee...,401.855974
3,3,jingle sleigh and workshop sing the holly naug...,577.885065
4,4,eggnog and fruitcake we wish from you merry no...,208.480133
5,5,poinsettia yuletide eggnog fruitcake gingerbre...,88.276042


In [16]:
# 各セルに関数を適用
sub_df["text"] = sub_df["text"].astype(str)
sub_df = sub_df.applymap(add_quotes_to_strings)

/tmp/ipykernel_114/428122046.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  sub_df = sub_df.applymap(add_quotes_to_strings)


In [17]:
sub_df[["id", "text"]].to_csv(f"{path}/out/submission_{ex_num}.csv", index=False, header=True, quoting=csv.QUOTE_NONE)
sub_df.to_csv(f"{path}/out/score_{ex_num}.csv", index=False, header=True, quoting=csv.QUOTE_NONE)

In [18]:
np.mean(sub_df["best_value"])

np.float64(415.94398656888114)